In [1]:
#! python3
## import gene panels from Genomics England website

import pandas as pd
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
## Selenium webdriver options
options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")
DRIVER_PATH = '/Users/home/.wdm/drivers/chromedriver/mac64/95.0.4638.54/chromedriver'

In [ ]:
urls_to_query = [
    "https://panelapp.genomicsengland.co.uk/panels/902/",
    "https://panelapp.genomicsengland.co.uk/panels/903/",
]

genomics_england_scrape_result = pd.DataFrame()

driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)

for url in urls_to_query:
    try:
        driver.get(url)
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        if "nhsgms" in str(url):
            div = soup.select_one("#gatsby-focus-wrapper > div.w-3\/4.mt-20.mx-auto > div > table")
        else:
            div = soup.select_one("#entities_table")
        table = pd.read_html(str(div), header=0)
        print("Successfully scraped", url)
        genomics_england_scrape_result = pd.concat([genomics_england_scrape_result, table[0]])
    except Exception as e:
        print("Unable to scrape", url, "because:", e) ###, opening in browser...")

driver.quit()

In [ ]:
genomics_england_scrape_result.to_csv('ckd.csv')

In [ ]:
gene_df = pd.read_csv('ckd_filtered.csv')
gene_df = gene_df.drop("Unnamed: 0", axis=1)
gene_df["Entity"] = gene_df["Entity"].str.split(" ").str[0]
gene_df = gene_df.drop_duplicates(["Entity"])
gene_df

In [ ]:
gcep_list = (""
).split(", ")
gene_list = gene_df["Entity"].to_list()

for gene in gcep_list:
    if gene not in gene_list:
        print(f"{gene} not in gene list")
        gene_df = gene_df.append([{"Entity":gene, "Mode of inheritance":""}])

gene_df = gene_df.sort_values(by="Entity")
gene_df = gene_df.reset_index()
gene_df = gene_df.drop("index", axis=1)
gene_df.to_csv('ckd_gene_list.csv')
gene_df